# UAC Bypass via Event Viewer
Detects UAC bypass method using Windows event viewer

## Rule Content
```
- title: UAC Bypass via Event Viewer
  id: 7c81fec3-1c1d-43b0-996a-46753041b1b6
  status: experimental
  description: Detects UAC bypass method using Windows event viewer
  references:
  - https://enigma0x3.net/2016/08/15/fileless-uac-bypass-using-eventvwr-exe-and-registry-hijacking/
  - https://www.hybrid-analysis.com/sample/e122bc8bf291f15cab182a5d2d27b8db1e7019e4e96bb5cdbd1dfe7446f3f51f?environmentId=100
  author: Florian Roth
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    methregistry:
      EventID: 13
      TargetObject: HKEY_USERS\\*\mscfile\shell\open\command
    methprocess:
      EventID: 1
      ParentImage: '*\eventvwr.exe'
    filterprocess:
      Image: '*\mmc.exe'
    condition: methregistry or ( methprocess and not filterprocess )
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.defense_evasion
  - attack.privilege_escalation
  - attack.t1088
  - car.2019-04-001
  falsepositives:
  - unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"13" AND registry_key_path.keyword:HKEY_USERS\\*\\mscfile\\shell\\open\\command) OR ((event_id:"1" AND process_parent_path.keyword:*\\eventvwr.exe) AND (NOT (process_path.keyword:*\\mmc.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()